In [50]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from matplotlib import image

from numpy import asarray

In [51]:
IMAGE_SIZE=256
BATCH_SIZE=32
TRAINING_SIZE=0.8
VALIDATION_SIZE=0.1
TEST_SIZE=0.1
CHANNEL=3
EPOCHS=50

In [52]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [53]:
class_names=dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [54]:
def split_data_to_Partion(dataset,TRAINING_SIZE,VALIDATIION_SIZE,TEST_SIZE):
    train_ds=dataset.take(int(len(dataset)*TRAINING_SIZE))
    validation_ds=dataset.skip(len(train_ds)).take(int(len(dataset)*VALIDATION_SIZE))
    teststartposition=len(train_ds)+len(validation_ds)
    test_ds=dataset.skip(teststartposition)

    return train_ds,validation_ds,test_ds

In [55]:
train_ds,validation_ds,test_ds=split_data_to_Partion(dataset,TRAINING_SIZE,VALIDATION_SIZE,TEST_SIZE)
print(len(train_ds),len(validation_ds),len(test_ds))

54 6 8


In [56]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [57]:
data_augmentation= tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [58]:
input_shape=(BATCH_SIZE,IMAGE_SIZE, IMAGE_SIZE,CHANNEL)
n=3
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape= input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n,activation='softmax'),
])
model.build(input_shape=input_shape)


In [59]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_11 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 sequential_12 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 conv2d_18 (Conv2D)          (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPooli  (32, 127, 127, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (32, 62, 62, 64)          0         
 ng2D)                                               

In [60]:
model.compile(
optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
metrics=['accuracy']
)

In [ ]:
history= model.fit(
train_ds,
epochs=EPOCHS,
batch_size=BATCH_SIZE,
verbose=1,
validation_data=validation_ds
)

Epoch 1/50
40/54 [=====================>........] - ETA: 25s - loss: 0.9091 - accuracy: 0.5031

In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss= history.history['val_loss']

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc, label='training accuracy')
plt.plot(range(EPOCHS), val_acc,label='validation accuracy')
plt.legend(loc='lower right')
plt.title('training and validation accuracy')

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss, label='training loss')
plt.plot(range(EPOCHS),val_loss, label='validation loss')
plt.legend(loc='upper right')
plt.title('training and validation loss')

In [ ]:
def predict(model,img):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array=tf.expand_dims(img_array,0)
    
    predictions=model.predict(img_array)
    
    predicted_class=class_names[numpy.argmax(predictions[0])]
    confidence=round(100*(numpy.max(predictions[0])),2)
    return predicted_class, confidence

In [ ]:
import numpy
plt.figure(figsize=(15,15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        predicted_class, confidence=predict(model,images[i].numpy())
    
        actual_class=class_names[labels[i]]
        plt.title(f"Atual:{actual_class}, \n Predicted:{predicted_class}.\n Confidence:{confidence}%")

In [ ]:
model_version=1
model.save(f"model/{model_version}")